In [11]:
from os import listdir, walk
from os.path import isfile, join
import numpy as np
from PIL import Image
import time
import cv2
from sys import getsizeof
from IPython.display import display
import random
import math
import datetime

import tensorflow as tf
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.layers import Input
from tensorflow.keras.utils import Sequence
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
#tf.config.optimizer.set_jit(True)
#tf.debugging.set_log_device_placement(True)
print(tf.config.experimental.list_physical_devices())

%run ./variables.ipynb
%run ./utils.ipynb

id_map = get_selected_taxons()
n_classes = len(list(id_map.keys()))

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'), PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Handle data

In [15]:
x_train, y_train, x_test, y_test = get_dataset(percentage_train = 0.9)
dataset_train = DiatomSequence(x_train, y_train, 50)

'185/185'

array([ 91, 105,  72, ..., 177, 168, 111])

# Model

In [ ]:
# fetching base model
base_model = Xception(include_top=False, weights='imagenet', input_shape=(256, 256, 3), pooling=None)

In [ ]:
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(185, activation='softmax')(x)
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
!rm -rf LOG_DIR
log_dir = LOG_DIR + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit(dataset_train, epochs=1, use_multiprocessing=True)

In [ ]:
dataset_train.__getitem__(0)[1]

In [ ]:
y_train